In [1]:
import os
import re
import glob
import random
import numpy as np
import scipy
import scipy.io as sio
import scipy.ndimage as ndimage
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patches as patches
from matplotlib.lines import Line2D
from PIL import Image
from ipywidgets import interact, interactive, fixed, interact_manual
import braingeneers
import braingeneers.data.datasets_electrophysiology as ephys
from braingeneers.analysis.analysis import SpikeData, read_phy_files
from matplotlib.colors import LinearSegmentedColormap

# New heading

In [9]:
def analyze_data(start, stop):
    dataset_path = '/home/jovyan/work/Human_Hippocampus/data/ephys/2023-04-02-e-hc328_unperturbed/derived/kilosort2/hc3.28_hckcr1_chip16835_plated34.2_rec4.2_curated.zip'
    
    sd = read_phy_files(dataset_path)
    sd_start = sd.subtime(start*1000, stop*1000)

    def calculate_mean_firing_rates(spike_data):
        mean_firing_rates = []
        for neuron_spikes in spike_data.train:
            num_spikes = len(neuron_spikes)
            time_duration = spike_data.length / 1000  # Assuming spike times are in milliseconds
            firing_rate = num_spikes / time_duration
            mean_firing_rates.append(firing_rate)

        return np.array(mean_firing_rates)
    
    firing_rates = calculate_mean_firing_rates(sd_start)
    
    def firing_plotter(sd):
        neuron_x = []
        neuron_y = []

        for neuron in sd.neuron_data[0].values():
            neuron_x.append(neuron['position'][0])
            neuron_y.append(neuron['position'][1])

        plt.figure(figsize=(8, 6))
        # Increase the 's' parameter to make the scatter points larger
        # Adjust the 'alpha' parameter to make the scatter points less opaque
        # Define a custom colormap going from pale pink (very low firing rate) to dark red (very high firing rate)
        colormap = LinearSegmentedColormap.from_list('custom_colormap', ['#FFB6C1', '#8B0000'], N=256)

        # Plot scatter points with adjusted colors based on firing rates using the custom colormap
        plt.scatter(neuron_x, neuron_y, s=7.5* firing_rates, c=firing_rates, cmap=colormap, alpha=0.7)
        
        plt.xlabel('um')
        plt.ylabel('um')
        plt.title(f"{start} sec.png")  # Adding the title

        # Set fixed limits for x and y axes
        plt.xlim(500, 1600)
        plt.ylim(0, 2200)
        
        plt.savefig(f"/home/jovyan/work/Human_Hippocampus/saved_plots/firing_animations/New_Data/{start}_sec.png")
        plt.close()
        
    firing_plotter(sd_start)

    return f"/home/jovyan/work/Human_Hippocampus/saved_plots/firing_animations/New_Data/{start}_sec.png"

def create_animated_gif():
    # TODO: Make the directory a parameter you can set in the function
    
    # Define the directory path
    directory = '/home/jovyan/work/Human_Hippocampus/saved_plots/firing_animations/'

#     # Create a subdirectory to save the figures
#     figures_directory = os.path.join(directory, f"dataset_{dataset_number}")
#     if not os.path.exists(figures_directory):
#         os.makedirs(figures_directory)
    figures_directory = "/home/jovyan/work/Human_Hippocampus/saved_plots/firing_animations/New_Data/"
    dataset_path = '/home/jovyan/work/Human_Hippocampus/data/ephys/2023-04-02-e-hc328_unperturbed/derived/kilosort2/hc3.28_hckcr1_chip16835_plated34.2_rec4.2_curated.zip'
    
    sd = read_phy_files(dataset_path)
    length = int(sd.length/1000) 
#     length = int(50)
    
    # Iterate over each second of the data
    for second in range(length):
        start = second
        stop = start + 1

        # Call analyze_data function
        image_path = analyze_data(start, stop)
        print(f"Generated image: {image_path}")

    # Directory path where the PNG files are located
    directory = figures_directory

    # Create a list of file names in the directory
#     file_list = sorted([filename for filename in os.listdir(directory) if filename.endswith('.png')], key=lambda x: int(re.search(r'\d+', x).group()))
    file_list = sorted([filename for filename in os.listdir(directory) if filename.endswith('.png')], key=lambda x: int(x.split("_")[0]))
    file_list = sorted([filename for filename in file_list if re.match(r'\d+(\.\d+)?_sec\.png', filename)], key=lambda x: float(re.search(r'\d+(\.\d+)?', x).group()))

    # Create a list to store the image frames
    frames = []

    # Iterate over each file and add it to the frames list
    for filename in file_list:
        # Create the full file path
        file_path = os.path.join(directory, filename)

        # Open the image file and append it to the frames list
        image = Image.open(file_path)
        frames.append(image)

    # Save the frames as an animated GIF
    save_path = "/home/jovyan/work/Human_Hippocampus/saved_plots/firing_animations/New_Data_Full/new_data_full.gif"
    frames[0].save(save_path, format='GIF', append_images=frames[1:], save_all=True, duration=500, loop=0)


In [10]:
# Example usage
create_animated_gif()

/tmp/ipykernel_721067/1568079650.py:64: DeprecationWarning: Call to deprecated function (or staticmethod) read_phy_files. (Prefer load_spike_data()) -- Deprecated since version 0.1.13.
  sd = read_phy_files(dataset_path)
/tmp/ipykernel_721067/1568079650.py:4: DeprecationWarning: Call to deprecated function (or staticmethod) read_phy_files. (Prefer load_spike_data()) -- Deprecated since version 0.1.13.
  sd = read_phy_files(dataset_path)
/tmp/ipykernel_721067/1568079650.py:23: DeprecationWarning: Call to deprecated function (or staticmethod) neuron_data. (Use NeuronAttributes instead of neuron_data, with the function load_spike_data())
  for neuron in sd.neuron_data[0].values():


Generated image: /home/jovyan/work/Human_Hippocampus/saved_plots/firing_animations/New_Data/0_sec.png


/tmp/ipykernel_721067/1568079650.py:4: DeprecationWarning: Call to deprecated function (or staticmethod) read_phy_files. (Prefer load_spike_data()) -- Deprecated since version 0.1.13.
  sd = read_phy_files(dataset_path)
/tmp/ipykernel_721067/1568079650.py:23: DeprecationWarning: Call to deprecated function (or staticmethod) neuron_data. (Use NeuronAttributes instead of neuron_data, with the function load_spike_data())
  for neuron in sd.neuron_data[0].values():


Generated image: /home/jovyan/work/Human_Hippocampus/saved_plots/firing_animations/New_Data/1_sec.png


/tmp/ipykernel_721067/1568079650.py:4: DeprecationWarning: Call to deprecated function (or staticmethod) read_phy_files. (Prefer load_spike_data()) -- Deprecated since version 0.1.13.
  sd = read_phy_files(dataset_path)
/tmp/ipykernel_721067/1568079650.py:23: DeprecationWarning: Call to deprecated function (or staticmethod) neuron_data. (Use NeuronAttributes instead of neuron_data, with the function load_spike_data())
  for neuron in sd.neuron_data[0].values():
/tmp/ipykernel_721067/1568079650.py:4: DeprecationWarning: Call to deprecated function (or staticmethod) read_phy_files. (Prefer load_spike_data()) -- Deprecated since version 0.1.13.
  sd = read_phy_files(dataset_path)


...

Generated image: /home/jovyan/work/Human_Hippocampus/saved_plots/firing_animations/New_Data/395_sec.png


/tmp/ipykernel_721067/1568079650.py:4: DeprecationWarning: Call to deprecated function (or staticmethod) read_phy_files. (Prefer load_spike_data()) -- Deprecated since version 0.1.13.
  sd = read_phy_files(dataset_path)
/tmp/ipykernel_721067/1568079650.py:23: DeprecationWarning: Call to deprecated function (or staticmethod) neuron_data. (Use NeuronAttributes instead of neuron_data, with the function load_spike_data())
  for neuron in sd.neuron_data[0].values():
/tmp/ipykernel_721067/1568079650.py:4: DeprecationWarning: Call to deprecated function (or staticmethod) read_phy_files. (Prefer load_spike_data()) -- Deprecated since version 0.1.13.
  sd = read_phy_files(dataset_path)


Generated image: /home/jovyan/work/Human_Hippocampus/saved_plots/firing_animations/New_Data/396_sec.png


/tmp/ipykernel_721067/1568079650.py:23: DeprecationWarning: Call to deprecated function (or staticmethod) neuron_data. (Use NeuronAttributes instead of neuron_data, with the function load_spike_data())
  for neuron in sd.neuron_data[0].values():


Generated image: /home/jovyan/work/Human_Hippocampus/saved_plots/firing_animations/New_Data/397_sec.png
